# COCO data preprocessing

This code will download the caption anotations for coco and preprocess them into an hdf5 file and a json file. 

These will then be read by the COCO data loader in Lua and trained on.

In [1]:
# lets download the annotations from http://mscoco.org/dataset/#download
import os
os.system('wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip') # ~19MB

0

In [2]:
os.system('unzip captions_train-val2014.zip')

0

In [2]:
import json
val = json.load(open('Data/final_data_val.json', 'r'))
infoOfVal = json.load(open('Data/final_data_val.json', 'r'))
infoOfVal = infoOfVal['images']
val['images'] = infoOfVal
#train = json.load(open('annotations/captions_train2014.json', 'r'))

In [3]:
print( val.keys() )
#print( val['info'] )
print( len(val['images']) )
print( len(val['annotations']) )
#print( val['images'][0] )
print( val['annotations'][0] )

dict_keys(['images', 'annotations'])
39792
39792
{'caption': ['गंदगी सड़क पर एक छोटा सा मोपेड पर लाल हेलमेट वाले एक आदमी', 'मनुष्य ग्रामीण इलाकों में एक गंदगी सड़क पर मोटर बाइक चला रहा है।', 'एक मोटरसाइकिल के पीछे की सवारी वाली एक आदमी', 'मोटर बाइक पर एक युवा व्यक्ति के साथ एक गंदगी पथ एक सब्जी क्षेत्र के पुल के साथ एक पुल और बादल-पुष्कृत पहाड़ों की पृष्ठभूमि के साथ स्थित है।', 'एक लाल शर्ट में एक आदमी और एक लाल टोपी एक पहाड़ी की ओर एक मोटरसाइकिल पर है।'], 'image_id': 391895}


In [8]:
import json
import os

# combine all images and annotations together
imgs = val['images']#+ train['images']
annots = val['annotations']# + train['annotations']

# for efficiency lets group annotations by image
itoa = {}
for a in annots:
    imgid = a['image_id']
    if not imgid in itoa: itoa[imgid] = []
    itoa[imgid].append(a)

# create the json blob
out = []
for i,img in enumerate(imgs):
    imgid = img['id']
    
    # coco specific here, they store train/val images separately
    loc = 'train2014' if 'train' in img['file_name'] else 'val2014'
    
    jimg = {}
    jimg['file_path'] = os.path.join(loc, img['file_name'])
    jimg['id'] = imgid
    
    sents = []
    annotsi = itoa[imgid]
    for a in annotsi:
        sents.append(a['caption'])
    jimg['captions'] = sents
    out.append(jimg)
    
json.dump(out, open('/home/himanshu/DL/caption/hindi-captioning/Data/coco_raw.json', 'w'))

In [9]:
# lets see what they look like
print(out[0])

{'file_path': 'val2014/COCO_val2014_000000391895.jpg', 'id': 391895, 'captions': [['गंदगी सड़क पर एक छोटा सा मोपेड पर लाल हेलमेट वाले एक आदमी', 'मनुष्य ग्रामीण इलाकों में एक गंदगी सड़क पर मोटर बाइक चला रहा है।', 'एक मोटरसाइकिल के पीछे की सवारी वाली एक आदमी', 'मोटर बाइक पर एक युवा व्यक्ति के साथ एक गंदगी पथ एक सब्जी क्षेत्र के पुल के साथ एक पुल और बादल-पुष्कृत पहाड़ों की पृष्ठभूमि के साथ स्थित है।', 'एक लाल शर्ट में एक आदमी और एक लाल टोपी एक पहाड़ी की ओर एक मोटरसाइकिल पर है।']]}
